In [11]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
import dlib
import numpy as np

device = "cuda:0" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", load_in_8bit=True, device_map={"": 0}, torch_dtype=torch.float16
)  # doctest: +IGNORE_RESULT

url = "http://images.cocodataset.org/val2017/000000039769.jpg"

frame = (Image.open('/mnt/data/tarun/celeba_hq_256/00000.jpg'\))
image=np.array(frame)
##detector = dlib.get_frontal_face_detector()
##face = detector(image)
##x, y, w, h = face[0].left(), face[0].top(), face[0].width(), face[0].height()
##cropped_frame = frame.crop((x, y, x+w, y+h))

#probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities


inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

generated_ids = model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB (GPU 0; 23.69 GiB total capacity; 16.90 GiB already allocated; 54.25 MiB free; 17.64 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

generated_ids = model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

/home/sushanth/anaconda3/envs/llt/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a man in a green shirt smiling at the camera


In [1]:
import os
from PIL import Image
import torch
import transformers
import json
import numpy as np
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

# Check if GPU is available and use it if so
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load the BLIP-2 processor and model
processor = transformers.Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = transformers.Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", load_in_8bit=True, device_map={"": 0}, torch_dtype=torch.float16
)
#model.to(device)

# Function to generate captions for an image
def generate_caption(image_path):
    image = Image.open(image_path)
    image = np.array(image)  # Convert to NumPy array
    #inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
    #generated_ids = model.generate(**inputs)
    #generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

    generated_ids = model.generate(**inputs)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    return generated_text

# Path to your image directory (replace with your actual path)
image_dir = "/mnt/data/tarun/celeba_hq_256"

# Initialize an empty dictionary to store image-caption pairs
captions = {}
i=0
# Loop through all images in the directory
for filename in os.listdir(image_dir):
    if filename.endswith((".jpg", ".jpeg", ".png")): 
        i=i+1 # Check for image extensions
        if i < 10000:
            image_path = os.path.join(image_dir, filename)
            caption = generate_caption(image_path)
            captions[image_path] = caption

# Save the captions to a JSON file
with open("captionsl.json", "w") as outfile:
    json.dump(captions, outfile, indent=4)

print("Captions saved successfully to captionsl.json!")


2024-02-24 06:39:37.900510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-24 06:39:38.367210: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-24 06:39:39.561527: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-24 06:39:39.561575: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sushanth/anaconda3/envs/llt/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

: 

In [5]:
from PIL import Image
import requests
from transformers import AutoTokenizer, BlipForConditionalGeneration

# Load the tokenizer and conditional generation model
tokenizer = AutoTokenizer.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# Preprocess the image
inputs = tokenizer(text="a photo of something", images=image, return_tensors="pt")

# Generate captions
outputs = model.generate(**inputs)

# Decode the generated captions
captions = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the captions
print(captions)


TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'images'